# 스파크 간단히 살펴보기

## 스파크의 기본 아키텍처
- 보통 한 대의 컴퓨터는 대규모 정보를 연산할 만한 자원이나 성능을 가지지 못함
  - 연산을 할 수 있다고 해도 완료하는데 너무 많은 시간이 걸림

- 컴퓨터 클러스터는 <strong>여러 컴퓨터의 자원을 모아 하나의 컴퓨터처럼 사용</strong>할 수 있게 만듦

- 이 <strong>클러스터에서 작업을 조율</strong>할 수 있는 프레임워크가 필요 <strong>-> 스파크</strong>가 그런 역할을 하는 프레임워크

- 스파크가 연산에 사용할 클러스터는 <strong>클러스터 매니저</strong>에서 관리함
  - Standalone
  - 하둡 YARN
  - Mesos

- 작업 처리 과정
  1. 사용자는 클러스터 매니저에 스파크 애플리케이션을 제출(submit)함
  2. 이를 제출받은 클러스터 매니저는 애플리케이션 실행에 필요한 자원을 할당
  3. 할당받은 자원으로 작업 처리
  
<img src="https://t1.daumcdn.net/cfile/tistory/991779375DEA85D00B"/>

## 스파크 애플리케이션

<img src="https://t1.daumcdn.net/cfile/tistory/99916B495DEA927C01"/>

- 스파크 애플리케이션은 <strong>드라이버 프로세스</strong>와 다수의 <strong>익스큐터 프로세스</strong>로 구성
  - 드라이버 프로세스는 클러스터 노드 중 하나에서 실행되며, main()함수 실행
    - 스파크 언어 API를 통해 다양한 언어로 실행 가능
  - 익스큐터 프로세스는 스파크 코드를 실행

- 드라이버 프로세스의 역할
  - 스파크 애플리케이션의 심장 같은 존재로, 애플리케이션 수명 주기 동안 관련 정보를 모두 유지함
  - 사용자 프로그램이나 입력에 대한 응답
  - 전반적인 익스큐터 프로세스의 작업과 관련된 분석, 배포, 스케줄링
  
- 익스큐터 드라이버의 역할
  - 드라이버 프로세스가 할당한 작업을 수행
    - 즉, 할당한 코드를 실행하고, 
    - 진행 상황을 다시 드라이버 노드에 보고

## 스파크의 다양한 언어 API
- 스파크의 언어 API를 이용하면 다양한 프로그래밍 언어로 스파크 코드를 실행할 수 있다.
  - 스파크는 스칼라로 개발
  - 스파크 언어 API를 통해 클러스터 머신에서 실행되는 스파크 코드로 변환
  
- 스파크는 사용자를 대신해 <strong>파이썬이나 R로 작성한 코드를 익스큐터의 JVM에서 실행할 수 있는 코드로 변환함</strong>

## SparkSession
- 스파크 애플리케이션은 <strong>SparkSession이라 불리는 드라이버 프로세스로 제어</strong>한다.
- SparkSession 인스턴스는 사용자가 정의한 처리 명령을 클러스터에서 실행한다.
- 하나의 SparkSession은 하나의 스파크 애플리케이션에 대응한다.

In [0]:
#spark 변수로 SparkSession 사용 가능
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
myRange = spark.range(1000).toDF('number')

In [0]:
display(myRange.limit(10))

number
0
1
2
3
4
5
6
7
8
9


----------
- 위에서 생성한 DataFrame은 한 개의 column과 1000개의 row로 구성
  - 할당된 숫자들(0~999)은 <strong>분산 컬렉션(RDD)</strong>을 나타냄
  - 숫자 범위의 각 부분이 <strong>서로 다른 익스큐터에 할당됨</strong>

## DataFrame

- 가장 대표적인 구조적 API
- 테이블의 데이터를 <strong>row와 column</strong>으로 단순하게 표현
- <strong>스키마</strong>: column과 column의 타입을 정의한 목록
- <strong>수천 대의 컴퓨터에 분산되어 있음</strong>
  - 단일 컴퓨터에 저장하기엔 데이터가 너무 크거나 계산이 너무 오래 걸릴 수 있기에
  
  
*스파크는 Dataset, DataFrame, SQL 테이블, RDD라는 몇 가지 핵심 추상화 개념을 갖고 잇음-> 모두 분산 데이터 모음을 표현함

### 파티션
- 스파크는 <strong>모든 익스큐터가 병렬로 작업을 수행</strong>할 수 있도록 파티션이라 불리는 청크 단위로 <strong>데이터를 분할</strong>함
- <strong>파티션: 클러스터의 물리적 머신에 존재하는 로우의 집합</strong>
  - 만약 파티션이 하나라면 스파크에 수천 개의 익스큐터가 있더라도 병렬성은 1
  - 그리고 수백 개의 파티션이 있더라도 익스큐터가 하나라면 병렬성은 1
- DataFrame의 파티션은 실행 중에 데이터가 컴퓨터 클러스터에서 물리적으로 분산되는 방식
- DataFrame을 사용하면 파티션을 수동 혹은 개별적으로 처리할 필요가 없다.
  - 물리적 파티션에 데이터 변환용 함수를 지정하면 스파크가 실제 처리 방법을 결정한다.

##  트렌스포메이션
- 스파크의 핵심 데이터 구조는 <strong>불변성(immutable)</strong>을 가짐
- 데이터 구조를 변경하려면 <strong>변경 방법을 스파크에게 알려줘야함</strong>
- 이때 사용하는 <strong>명령을 트랜스포메이션</strong>이라함

In [0]:
#짝수를 찾는 간단한 트랜스포메이션 예제
tmp = myRange.where('number>0 and number%2=0')

-----
- 위 코드를 실행해도 결과는 출력되지 않음
- <strong>추상적인 트랜스포메이션만 지정한 상태</strong>이기 때문에 <strong>액션</strong>을 호출하지 않으면 스파크는 <strong>실제 트랜스포메이션</strong>을 수행하지 않음
- 트랜스포메이션에는 두 가지 유형이 있음
  - 좁은 의존성(narrow dependency)
  - 넓은 의존성(wide dependency)

### 좁은 의존성

<img src="https://www.programmersought.com/images/12/f59d1f9135f1df04d7d7a2005637add4.png"/>

- <strong>각 입력 파티션이 하나의 출력 파티션</strong>에만 영향을 미치는 것
- 위 예제에서 where 구문은 좁은 의존성을 가짐
- 파이프라이닝을 자동으로 수행함
  - 모든 작업이 메모리에서 일어남

### 넓은 의존성
<img src="https://www.programmersought.com/images/972/ac3487a04dc258a5a6aaf4e5fe62a8dc.png"/>

- <strong>하나의 입력 파티션이 여러 출력 파티션</strong>에 영향을 미침
- 스파크가 클러스터에서 파티션을 교환하는 <strong>셔플</strong>이 수행됨
  - 셔플의 결과를 디스크에 저장함

## 지연 연산
- <strong>스파크가 연산 그래프를 처리하기 직전까지 기다리는 동작 방식</strong>
  - 스파크는 연산 명령이 내려진 즉시 데이터를 수정하지 않음
  - 원시 데이터에 적용할 트랜스포메이션의 <strong>논리적 실행 계획</strong>을 생성함
  - 코드를 실행하는 마지막 순간까지 대기하다가 트랜스포메이션을 간결한 <strong>물리적 실행 계획</strong>으로 컴파일함
- 이를 통해 전체 데이터 흐름을 <strong>최적화</strong>할 수 있는 것임

## 액션
- <strong>실제 연산을 수행</strong>하기위해 액션 명령을 내림
- 액션은 일련의 트랜스포메이션으로부터 결과를 계산하도록 지시하는 것
- 액션의 세 가지 유형
  - 콘솔에서 데이터를 <strong>보는</strong> 액션
  - 각 언어로 된 네이티브 객체에 데이터를 <strong>모으는</strong> 액션
  - 출력 데이터소스에 <strong>저장</strong>하는 액션

In [0]:
#가장 단순한 액션인 count메서드 -> dataframe의 전체 레코드 수를 반환함
tmp.count()

Out[ 10 ]: 499

-----
- 위처럼 액션을 지정하면 스파크 job이 시작됨
- 스파크가 제공하는 스파크 UI로 클러스터에서 실행 중인 스파크 job 모니터링 가능

## 스파크 UI
- 스파크 job의 진행 상황을 모니터링할 때 사용
  - 스파크 UI는 드라이버 노드의 4040 포트로 접속 가능
  - 로컬: http://localhost:4040
- 확인 정보
  - 스파크 job 상태
  - 환경 설정
  - 클러스터 상태
  - etc.
- 핵심 요약(디테일한 내용은 나중에)
  - <strong>스파크 job은 개별 액션에 의해 트리거되는 다수의 트랜스포메이션</strong>으로 이루어져 있으며,
  - <strong>스파크 UI로 job을 모니터링</strong>할 수 있다.